In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

## Install libraries

In [ ]:
from IPython.display import display, Javascript

display(Javascript('''
(function() {
    var interval = setInterval(function() {
        if (typeof google !== 'undefined' && google.translate && google.translate.TranslateElement) {
            clearInterval(interval);
            google.translate.TranslateElement = function() {};
            document.getElementById('google_translate_element')?.remove();
        }
    }, 1000);
})();
'''))

```bash
conda create -n edu4 python=3.11 jupyter matplotlib
```

```bash 
! pip install -U -r requirements.txt
```

```bash
! pip install -U numpy
! pip install -U scikit-learn
```

## Update repository

In [ ]:
! git pull

## Add import path

In [ ]:
import os
import sys
import gc

In [ ]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
del module_path

## Organize imports

In [ ]:
import multiprocessing
from pathlib import Path

In [ ]:
import json

In [ ]:
from collections import OrderedDict

In [ ]:
from functools import reduce

In [ ]:
import seaborn as sns

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

In [ ]:
from tqdm import tqdm

In [ ]:
import plotly.express as px

In [ ]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.discriminant_analysis import (
    LinearDiscriminantAnalysis, 
    QuadraticDiscriminantAnalysis
)
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier 
from sklearn.datasets import (
    load_iris,
    load_wine
)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import (
    MaxAbsScaler,
    MinMaxScaler,
    StandardScaler
)
from sklearn.metrics import (
    classification_report,
    confusion_matrix
)

In [ ]:
from scipy import stats
from scipy.interpolate import interp1d

In [ ]:
import torch

In [ ]:
from torch import nn
from torchvision import transforms
import torchvision.transforms.functional as F
from torchvision.utils import make_grid
from torchvision.datasets import MNIST

In [ ]:
from captum.attr import (
    IntegratedGradients, 
    LayerIntegratedGradients,
    NeuronGradient,
    NeuronIntegratedGradients,
    NeuronGuidedBackprop,
    NeuronDeepLift,
    NeuronDeepLiftShap,
    NeuronGradientShap,
)
from captum.attr import visualization as viz

#### Number of CPU cores

In [ ]:
workers = multiprocessing.cpu_count()
workers

In [ ]:
SEED = 2024

## Initialize Path

In [ ]:
PATH = Path('data')
model_dir = PATH / 'models'
model_path = model_dir / 'simple_cnn_mnist_model.ckpt'
config_dir = PATH / 'config'
config_dir.mkdir(exist_ok=True, parents=True)
config_1_layer_path = config_dir / 'neurons_1_layer.json'
config_cnn_layer_1 = config_dir / 'neurons_cnn_1_layer.json'
images_dir = PATH / 'images'
images_dir.mkdir(exist_ok=True, parents=True)
pumpkin_path = PATH / 'Pumpkin_Seeds_Dataset.xlsx'

## Model wrapper

In [ ]:
class NetWrapper(object):

    def __init__(self, net, transform):
        self._net = net.eval()
        self.transform = transform

    @property
    def net(self):
        return self._net
    
    def __getitem__(self, i):
        return self.net[i]

    def __len__(self):
        return len(self.net)

    @torch.inference_mode()
    def forward(self, x, k=6):
        t = self.transform(x)
        r = self[: k](t)

        return r        
    
    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)

In [ ]:
def find_device():
    if torch.cuda.is_available():
        device = torch.device('cuda')
    elif torch.backends.mps.is_available():
        device = torch.device('mps')
    else:
        device = torch.device('cpu')

    return device

## Load the model

In [ ]:
model = torch.load(model_path, map_location='cpu')

In [ ]:
def clear_state_dict(state_dict):
    for key in list(state_dict.keys()):
        state_dict[key.replace('model.0.', 'conv1.')] = state_dict.pop(key)
    for key in list(state_dict.keys()):
        state_dict[key.replace('model.3.', 'conv2.')] = state_dict.pop(key)
    for key in list(state_dict.keys()):
        state_dict[key.replace('model.8.', 'fc1.')] = state_dict.pop(key)
    for key in list(state_dict.keys()):
        state_dict[key.replace('model.11.', 'fc2.')] = state_dict.pop(key)

    return state_dict

In [ ]:
state_dict = clear_state_dict(model['state_dict'])

In [ ]:
channels = 1
width = 28
height = 28
hidden_size = 16
num_classes = 10
in_features = channels * width * height

In [ ]:
net = nn.Sequential(OrderedDict([
    ('conv1', nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)),
    ('act1', nn.ReLU()),
    ('mxp1', nn.MaxPool2d(kernel_size=2, stride=2)),
    ('conv2', nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)),
    ('act2', nn.ReLU()),
    ('mxp2', nn.MaxPool2d(kernel_size=2, stride=2)),
    ('flatten', nn.Flatten()),
    ('fc1', nn.Linear(64 * 7 * 7, 128)),
    ('act3', nn.ReLU()),
    ('fc2', nn.Linear(128, 10)),
]))

In [ ]:
net.load_state_dict(state_dict)

In [ ]:
net = net.eval()

## Helper functions

In [ ]:
def intersect(*arrs):
    return reduce(np.intersect1d, (arrs))

In [ ]:
def layer_V(data, net, k=5):
    V = list()
    X = list()
    with tqdm(data) as ds:
        for i, (x, y) in enumerate(ds):
            v = net(x, k=k).detach().numpy()
            V.append(v)
            X.append(x)

    return np.array(V), X

In [ ]:
def loop_maxes(V, func, *args, **kwargs):
    with tqdm(V) as mstml:
        for i, v in enumerate(mstml):
            func(i, v, *args, **kwargs)

In [ ]:
def select_top(V, idx, thresh):
    tops = list()
    def add_to_top(i, v):
        if thresh <= v[idx]:
            tops.append(i)
    loop_maxes(V, lambda i, v: add_to_top(i, v))
    
    return tops

In [ ]:
def find_v_x(V, mrng, idx):
    mid = np.argmin(np.array(V)[mrng], axis=0)[idx]
    x_id = mrng[mid]
    v_x = V[x_id]

    return v_x, x_id

In [ ]:
def find_v_A(V, mrng):
    return np.minimum.reduce(np.array(V)[mrng])

In [ ]:
def find_G_x(V, v_x):
    with tqdm(V) as mstm:
        G_x = np.array([i for i, v in enumerate(mstm) if np.all(v_x <= v)])

    return G_x

In [ ]:
def show_img(ds, idx):
    plt.imshow(ds[idx][0])

In [ ]:
def show(imgs, h=12, w=12):
    if not isinstance(imgs, list):
        imgs = [imgs]
    fig, axs = plt.subplots(
        ncols=len(imgs),
        figsize=(w, h),
        squeeze=False
    )
    for i, img in enumerate(imgs):
        img = img.detach()
        img = F.to_pil_image(img)
        axs[0, i].imshow(np.asarray(img))
        axs[0, i].set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])

In [ ]:
def show_grid(G_A, data, nrow=8, h=12, w=12, my=None):
    G_A_F = G_A.ravel()
    to_tensor = transforms.ToTensor()
    A_gr = [
        to_tensor(data[i][0]) for i in G_A_F
    ] if my is None else [
        to_tensor(data[i][0]) for i in G_A_F if data[i][1] != my
    ]
    grid = make_grid(A_gr, nrow=nrow)
    show(grid, h=h, w=w)

In [ ]:
def vis_features(
        *n_idxs, 
        arg_max=None, 
        data=None, 
        layer=3, 
        func=NeuronIntegratedGradients, 
        use_plot=False, 
        **kwargs
):
    # Assuming model is your neural network and input_img is the input image tensor
    # Define the target layer and neuron index
    target_layer = wnet[:layer]
    # Initialize Integrated Gradients
    neuron_ig = func(wnet.net, wnet.net[layer])

    attr_igs = list()
    input_imgs = list()
    arg_max_fl = arg_max.ravel()
    for n_idx in n_idxs:
        # Compute the attributions for the target neuron
        input_img = data[arg_max_fl[n_idx]][0]
        input_tns = transform(input_img)
        target_neuron_index = n_idx
        attributions_ig = neuron_ig.attribute(
            input_tns, 
            neuron_selector=target_neuron_index,
            **kwargs,
        )
        attr_ig = attributions_ig.squeeze().cpu().detach().numpy()
        attr_igs.append(attr_ig)
        input_imgs.append(input_img)

    # fig, ax = plt.subplots(len(n_idxs), 2, figsize=(10, 5))

    if use_plot:
        for r, (attr_ig, input_img) in enumerate(zip(attr_igs, input_imgs)):
            # Display the first image
            fig, ax = plt.subplots(1, 2, figsize=(10, 5))
            ax[0].imshow(input_img, cmap='gray')
            ax[0].axis('off')  # Hide the axes
            
            # Display the second image
            ax[1].imshow(attr_ig, cmap='gray')
            ax[1].axis('off')  # Hide the axes
        
        # Show the plot
        plt.show()

    return attr_igs

In [ ]:
def display_features(attrs, data, arg_max):
    rows = len(attrs)
    arg_max_fl = arg_max.ravel()
    # fig, axes = plt.subplots(rows, 3, figsize=(15, 5 * rows))
    for idx in range(rows):
        attr = np.expand_dims(attrs[idx], axis=2)
        orig = np.expand_dims(data[arg_max_fl[idx]][0], axis=2)
        titl = str(idx)
        fig, axs = viz.visualize_image_attr_multiple(
            attr, 
            orig, 
            [
                'original_image', 
                'heat_map', 
                'masked_image',
            ], 
            [
                'all', 
                'positive', 
                'positive'
            ],
            use_pyplot=True,
            titles=[titl, titl, titl],
            show_colorbar=True,
        )
        
    #     axes[idx, 0] = axs[0]
        
    #     axes[idx, 1] = axs[1]
        
    #     axes[idx, 2] = axs[2]

    # plt.tight_layout()
    # plt.show()


## Feature FCA analysis methods

In [ ]:
def features_hist(*n_Fs, V=np.zeros((1, 16))):
    rows = len(n_Fs)
    vs_ls = list()
    with tqdm(n_Fs) as pn_Fs:
        for n_F in pn_Fs:
            vs = [v[n_F] for v in V]
            vs_ls.append(vs)
    fig, axs = plt.subplots(rows, 1, sharey=True, tight_layout=True, figsize=(8 * rows, 32))
    for r in range(rows):
        vs_h = vs_ls[r]
        axs[r].hist(vs_h)
        axs[r].set_xlim(0, 32)  # Set the X-axis limit if you want a specific range
        axs[r].set_xticks(np.arange(0, 32, 0.5))  # Ensure the ticks match the new range
        # axs[r].set_title(str(vs_h))
    

In [ ]:
class LayerFCA(object):

    def __init__(self, V_X, U_X, data):
        self.V_X = V_X
        self.U_X = U_X
        self.data = data
        self.G_As = list()
        self.v_As = list()
        self.D = None
        self.v_D = None
        self.U_D = None
        self.G_U_D = None
        self.find_G_x = find_G_x
        self.find_v_A = find_v_A
        self.find_G_x = find_G_x

    def fca_v(self, ns, ths):
        for n_A, th_A in zip(ns, ths):
            G_A_v_A = select_top(self.V_X, n_A, th_A)
            v_A = find_v_A(self.V_X, G_A_v_A)
            self.v_As.append(v_A)
            G_A = find_G_x(self.V_X, v_A)
            self.G_As.append(G_A)
        self.D = intersect(*self.G_As) if self.G_As else []
        self.v_D = np.maximum.reduce(self.v_As)
        
        return self.D, self.v_D

    def fca_u(self, ns, ths):
        D, _ = self.fca_v(ns, ths)
        self.u_D = find_v_A(
            self.U_X, D
        ) if np.any(D) else np.zeros(
            (16,), dtype=float
        )
        self.G_u_D = find_G_x(self.U_X, self.u_D)

        return self.G_u_D

    @staticmethod
    def count_ys(ys):
        un, cn = np.unique(ys, return_counts=True)
        uncn = np.array([un, cn])

        return uncn

    def _report_u(self, G_u_D, data=None):
        data_ls = self.data if data is None else data
        ys = np.array([data_ls[idx][1] for idx in G_u_D])
        uncn = self.count_ys(ys)
        if data is None:
            self.uncn = uncn

        return uncn

    def report(self, G_u_D, data):
        return self._report_u(G_u_D, data=data)

    def fca_u_arr(self, ns_arr):
        ns = [nr[0] for nr in neurons[neur_idx]]
        ts = [nr[1] for nr in neurons[neur_idx]]
        G_U_D = self.fca_u(ns, ts)
        self._report_u(G_U_D)

        return self.G_u_D

    @staticmethod
    def G_U(U_X, u_D):
        return find_G_x(U_X, u_D)

    def find_u_G_u(self, v):
        G_v = find_G_x(self.V_X, v)
        u_D = find_v_A(self.U_X, G_v)
        G_u = find_G_x(self.U_X, u_D)
        self._report_u(G_u)

        return G_v, u_D, G_u


    def find_G_u(self, v, U, X):
        G_v, u_D, G_u = self.find_u_G_u(v)
        G_rest = find_G_x(U, u_D)

        return G_rest

## Initialize MNIST dataset

In [ ]:
transform = transforms.Compose(
            [
                transforms.ToTensor(),
                transforms.Normalize((0.1307,), (0.3081,)),
            ]
)

In [ ]:
data_train = MNIST(images_dir, train=True, download=True)
data_test = MNIST(images_dir, train=False, download=True)

In [ ]:
next(net.parameters()).device

In [ ]:
wnet = NetWrapper(net, transform)

In [ ]:
wnet.net

In [ ]:
wnet.net[:6]

In [ ]:
layer_V_n = 3
layer_U_n = 6

In [ ]:
V_X_train, X_V_train = layer_V(data_train, wnet, k=layer_V_n)

In [ ]:
V_X_test, X_V_test = layer_V(data_test, wnet, k=layer_V_n)

In [ ]:
U_X_train, X_U_train = layer_V(data_train, wnet, k=layer_U_n)

In [ ]:
U_X_test, X_U_test = layer_V(data_test, wnet, k=layer_U_n)

In [ ]:
arg_max = np.argmax(V_X_train, axis=0)
arg_max.shape

In [ ]:
arg_top = np.argsort(V_X_train, axis=0)
arg_top.shape

In [ ]:
i = 0

In [ ]:
arg_top[-10:].shape

In [ ]:
arg_top[-10:,:, :].shape

In [ ]:
V_X_train[arg_top[-100:,1, 9, 2]][:,1, 9, 2]

In [ ]:
show_grid(arg_top[-100:,1, 9, 2], data_train, nrow=32)

In [ ]:
np.max(V_X_train, axis=0)

In [ ]:
V_X_train[arg_max[13], :]

In [ ]:
show_grid(arg_max[1,:], data_train, nrow=14)

In [ ]:
show_grid(arg_max, data_train, nrow=14)

In [ ]:
attr_all = vis_features(
    *list(range(14)),
    arg_max=arg_max,
    data=data_train,
    # func=NeuronGradientShap,
    func=NeuronGuidedBackprop,
    # n_steps=n_steps,
)

In [ ]:
neuron_ig = NeuronGuidedBackprop(wnet.net, wnet.net[3])
input_img = data_train[arg_max[0, 0]][0]
input_tns = transform(input_img)
attributions_ig = neuron_ig.attribute(
    input_tns, 
    neuron_selector=0
)

In [ ]:
display_features(attr_all, data_train)

In [ ]:
len(attr_all)

In [ ]:
with config_1_layer_path.open('r') as js:
    neuron_js = json.load(js)
neurons = neuron_js['neurons']
len(neurons)

## Alanyze maximum stimulus

In [ ]:
neurons = np.zeros((32, 14, 14), dtype=float)
# neurons[9, 1] = 0.2
# neurons[9, 2] = 0.2
# neurons[2, 0] = 0.2
# neurons[7, 0] = 0.01
neurons[2, 7] = 0.0002
neurons[7, 2] = 0.16
neurons[8, 2] = 0.0001
neurons[9, 2] = 0.0001
neurons[10, 2] = 0.0002
neurons[10, 6] = 0.0002
neurons[10, 7] = 0.0002

In [ ]:
neurons = np.zeros((32, 14, 14), dtype=float)
neurons[1, 11, 7] = 0.8
neurons[1, 12, 7] = 0.8
neurons[1, 4, 10] = 0.8
neurons[1, 5, 10] = 0.8
neurons[1, 6, 10] = 0.8
neurons[1, 7, 9] = 0.8
neurons[1, 8, 9] = 0.8
# neurons[5, 2] = 0.28
# neurons[6, 1] = 0.2
# neurons[7, 1] = 0.01

In [ ]:
neurons = np.zeros((32, 14, 14), dtype=float)
neurons[1, 11, 7] = 0.4
neurons[1, 12, 7] = 0.4
neurons[1, 4, 10] = 0.4
neurons[1, 5, 10] = 0.4
neurons[1, 6, 10] = 0.4
neurons[1, 7, 9] = 0.4
neurons[1, 8, 9] = 0.4
# neurons[5, 2] = 0.28
# neurons[6, 1] = 0.2
# neurons[7, 1] = 0.01

In [ ]:
neurons = np.zeros((32, 14, 14), dtype=float)
neurons[1, 11, 7] = 0.2
neurons[1, 12, 7] = 0.2
neurons[1, 4, 10] = 0.2
neurons[1, 5, 10] = 0.2
neurons[1, 6, 10] = 0.2
neurons[1, 7, 9] = 0.2
neurons[1, 8, 9] = 0.2
# neurons[5, 2] = 0.28
# neurons[6, 1] = 0.2
# neurons[7, 1] = 0.01

In [ ]:
# neurons = np.zeros((32, 14, 14), dtype=float)
neurons[8, 3, 3] = 0.8
neurons[8, 3, 4] = 0.8
neurons[8, 3, 4] = 0.8
neurons[8, 3, 5] = 0.8
neurons[8, 3, 6] = 0.8
neurons[8, 3, 7] = 0.8
neurons[8, 3, 8] = 0.8
neurons[8, 3, 9] = 0.8
# neurons[5, 2] = 0.28
# neurons[6, 1] = 0.2
# neurons[7, 1] = 0.01

In [ ]:
# neur_idx = -1
# with config_1_layer_path.open('r') as js:
#     neuron_js = json.load(js)
# neurons = neuron_js['neurons']
v = neurons
layer_fca = LayerFCA(V_X_train, U_X_train, data_train)
G_v, u_D, G_u = layer_fca.find_u_G_u(v)
G_v_test = layer_fca.find_G_x(V_X_test, v)
G_u_test = layer_fca.find_G_x(U_X_test, u_D)

In [ ]:
G_u.shape, G_u_test.shape

In [ ]:
[G_v.shape, G_u.shape], np.max(u_D)

In [ ]:
uncn_test = layer_fca.report(G_U_test, data_test)

In [ ]:
G_u_test.shape, uncn_test

In [ ]:
layer_fca.uncn

In [ ]:
show_grid(G_v_test, data_test, nrow=32)

In [ ]:
show_grid(G_u_test, data_test, nrow=32)

In [ ]:
y_hs = [np.argmax(wnet(data_test[idx][0])) for idx in G_u_test]

In [ ]:
uncn_hat = layer_fca.count_ys(y_hs)

In [ ]:
uncn_hat

In [ ]:
net[3].weight[:, 0], net[3].bias

In [ ]:
layer_fca.uncn

In [ ]:
show_grid(G_u_test, data_test, nrow=48, h=64, w=64)

In [ ]:
show_grid(G_u_test, data_test, nrow=48, h=64, w=64, my=7)

## Visualization of distribution

In [ ]:
# features_hist(*list(zip(*neurons[neur_idx]))[0], V=V_X)

In [ ]:
def layer_hist(X_y, V_X, y=None):
    if y is None:
        V_X_y = V_X
    else:
        with tqdm(V_X) as p_V_X:
            V_X_y = np.array(
                [v_x for x_y, v_x in zip(X_y, p_V_X) if x_y[1] == y]
            )

    return V_X_y

In [ ]:
V_X_0 = layer_hist(data_train, V_X_train, y=0)
V_X_1 = layer_hist(data_train, V_X_train, y=1)
V_X_2 = layer_hist(data_train, V_X_train, y=2)
V_X_3 = layer_hist(data_train, V_X_train, y=3)
V_X_4 = layer_hist(data_train, V_X_train, y=4)

In [ ]:
V_X_4.shape

In [ ]:
ds = V_X_4[:, 0, 0]
ds.shape

In [ ]:
V_X_0[1, 1].shape

In [ ]:
np.unique(V_X_0[:, 0, 0], return_counts=True)

In [ ]:
st = 4
colors = [None, 'green', 'blue', 'purple']
for i in range(V_X_4.shape[1]):
    # print(f'V histograms for i = {i}')
    for j in range(0, V_X_4.shape[2], st):
        fig, axes = plt.subplots(1, st, figsize=(14, 6))
        for s in range(st):
            kwargs = dict(
                kde=False, 
                # binwidth=1,
                bins=np.linspace(-0.5, 0.5, 2),
                ax=axes[s],
            )
            colors = {'V_1': 'brown', 'V_2': 'green'}
            sns.histplot(V_X_0[:, i, min(j + s, 13)], **kwargs)
            # sns.histplot(V_X_1[i, min(j + s, 13)], ax=axes[s])
            # sns.histplot(V_X_2[i, min(j + s, 13)], ax=axes[s])
            # sns.histplot(V_X_3[i, min(j + s, 13)], ax=axes[s])
            # sns.histplot(V_X_4[i, min(j + s, 13)], ax=axes[s])
            axes[s].set_xlabel(f'0 - {i} {min(j + s, 13)}')
        plt.show()
# plt.show()

In [ ]:
# Function to visualize the activations
def visualize_activations(model, image, layers=[2, 4], hist=False):
    # Pass the image through the network
    activations = list()
    with torch.no_grad():
        for k in layers:
            output = model(image, k=k)
            activations.append(output.detach())

    # Plot the activations
    for k in range(0, 32, 16):
        fig, axes = plt.subplots(4, 4, figsize=(12, 12))
        for i, ax in enumerate(axes.flat):
            if i < 16:
                activation = activations[0][k + i].cpu()
                if hist:
                    ax = fig.add_subplot(4, 4, i + 1, projection='3d')
                    activation = activation.numpy()
                    x, y = np.meshgrid(np.arange(activation.shape[1]), np.arange(activation.shape[0]))
                    ax.bar3d(x.ravel(), y.ravel(), np.zeros_like(x.ravel()), 1, 1, activation.ravel(), shade=True)
                else:
                    im = ax.imshow(activation, cmap='viridis')
                    ax.set_title(f'Conv1 - Filter {k + i}')
                    ax.axis('off')
                # fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
        plt.show()
    print('======================')
    for k in range(0, 64, 16):
        fig, axes = plt.subplots(4, 4, figsize=(12, 12))
        for i, ax in enumerate(axes.flat):
            if i < 16:
                im = ax.imshow(activations[1][k + i].cpu(), cmap='viridis')
                ax.set_title(f'Conv2 - Filter {k + i}')
                ax.axis('off')
                fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
        plt.show()

    return activations

In [ ]:
wnet[:3]

In [ ]:
act1 = wnet(data_test[0][0], k=3)

In [ ]:
act1.shape

In [ ]:
i = 0
x, y = data_test[i]
y

In [ ]:
# Load an example image
example_image, _ = data_test[i]
# example_image = example_image.unsqueeze(0)  # Add batch dimension

# Visualize the activations
acts = visualize_activations(wnet, example_image, layers=[3, 6], hist=False)

In [ ]:
def show_activation(activation, layer_name='', filter_index=0):
    # Plot the activation as a grid of numbers
    fig, ax = plt.subplots(figsize=(16, 16))
    cax = ax.matshow(activation, cmap='viridis')

    # Customize the ticks to show each step
    ax.set_xticks(np.arange(activation.shape[1]))
    ax.set_yticks(np.arange(activation.shape[0]))

    # Show grid lines
    ax.grid(color='black', linestyle='-', linewidth=1, which='both')
    ax.xaxis.set_ticks_position('bottom')

    # Annotate the grid with the activation values
    for (i, j), val in np.ndenumerate(activation):
        ax.text(j, i, f'{val:.2f}', ha='center', va='center', color='white')

    fig.colorbar(cax)
    plt.title(f'Activation of {layer_name} - Filter {filter_index}')
    plt.show()

In [ ]:
show_activation(acts[0][1])

In [ ]:
show_activation(acts[0][8])

In [ ]:
fig, ax = plt.subplots()
cax = ax.matshow(acts[0][1], cmap='viridis')
fig.colorbar(cax)
plt.show()

In [ ]:
def visualize_weights(layer, num_filters=32):
    weights = layer.weight.data.cpu().numpy()
    for k in range(0, num_filters, 16):
        fig, axes = plt.subplots(4, 4, figsize=(12, 12))
        for i, ax in enumerate(axes.flat):
            if i < num_filters:
                img = weights[i, 0, :, :]  # Select the i-th filter and the first input channel
                im = ax.imshow(img, cmap='viridis')
                ax.axis('off')
                ax.set_title(f'Filter {i}')
                fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
        plt.show()

In [ ]:
wnet.net[3]

In [ ]:
# Visualize weights of the first convolutional layer
visualize_weights(wnet.net[0], num_filters=32)

# Visualize weights of the second convolutional layer
visualize_weights(wnet.net[3], num_filters=64)